## Second Attempt

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# 1. Caricamento dei dati scaricati manualmente
path_dati = "/Users/tommaso/Desktop/GT Data/Official"
nomi_gruppi = {
    "Gruppo1": "Termini_Diretti",
    "Gruppo2": "Alimentari",
    "Gruppo3": "Energia",
    "Gruppo4": "Abitazione",
    "Gruppo5": "Trasporti",
    "Gruppo6": "Politiche_Economiche",
    "Gruppo7": "Aspettative_Consumatori",
    "Gruppo8": "Sanita",
    "Gruppo9": "Ricreazione"
}

# Funzione per caricare i dati GT
def carica_dati_gt(path, nomi_file):
    dati_gruppi = {}
    
    for nome_file, nome_gruppo in nomi_gruppi.items():
        file_path = os.path.join(path, f"{nome_file}.csv")
        
        try:
            # Carica il CSV, assumendo che la prima colonna sia la data
            df = pd.read_csv(file_path)
            
            # Normalizza il nome delle colonne per sicurezza
            df.columns = [col.strip().lower() for col in df.columns]
            
            # Identifica la colonna della data (potrebbe essere chiamata 'date', 'time', 'week', ecc.)
            # Questa è una stima, potresti dover adattare in base al formato reale
            col_data = [col for col in df.columns if any(term in col.lower() for term in ['date', 'time', 'week', 'day', 'month', 'anno', 'data'])][0]
            
            # Imposta la colonna della data come indice
            df.set_index(col_data, inplace=True)
            
            # Converte l'indice in datetime se non lo è già
            df.index = pd.to_datetime(df.index)
            
            # Verifica che 'inflazione' sia presente
            if 'inflazione' not in df.columns:
                print(f"Attenzione: 'inflazione' non trovata in {nome_file}, questo potrebbe causare problemi di allineamento")
            
            dati_gruppi[nome_gruppo] = df
            print(f"✓ Dati caricati con successo per {nome_gruppo}")
            
        except Exception as e:
            print(f"✗ Errore nel caricamento di {nome_file}: {e}")
    
    return dati_gruppi

# Carica i dati
dati_gruppi = carica_dati_gt(path_dati, nomi_gruppi)




✗ Errore nel caricamento di Gruppo1.csv: [Errno 2] No such file or directory: '/Users/tommaso/Desktop/GT Data/Official/Gruppo1.csv.csv'
✗ Errore nel caricamento di Gruppo2.csv: [Errno 2] No such file or directory: '/Users/tommaso/Desktop/GT Data/Official/Gruppo2.csv.csv'
✗ Errore nel caricamento di Gruppo3.csv: [Errno 2] No such file or directory: '/Users/tommaso/Desktop/GT Data/Official/Gruppo3.csv.csv'
✗ Errore nel caricamento di Gruppo4.csv: [Errno 2] No such file or directory: '/Users/tommaso/Desktop/GT Data/Official/Gruppo4.csv.csv'
✗ Errore nel caricamento di Gruppo5.csv: [Errno 2] No such file or directory: '/Users/tommaso/Desktop/GT Data/Official/Gruppo5.csv.csv'
✗ Errore nel caricamento di Gruppo6.csv: [Errno 2] No such file or directory: '/Users/tommaso/Desktop/GT Data/Official/Gruppo6.csv.csv'
✗ Errore nel caricamento di Gruppo7.csv: [Errno 2] No such file or directory: '/Users/tommaso/Desktop/GT Data/Official/Gruppo7.csv.csv'
✗ Errore nel caricamento di Gruppo8.csv: [Errno 

In [ ]:
# 2. Allineamento delle scale usando "inflazione" come termine comune
def allinea_scale(dati_gruppi, query_riferimento="inflazione"):
    # Calcola la media del termine di riferimento in ogni gruppo
    medie_riferimento = {}
    for gruppo, dati in dati_gruppi.items():
        cols_lower = [col.lower() for col in dati.columns]
        if query_riferimento.lower() in cols_lower:
            idx = cols_lower.index(query_riferimento.lower())
            col_name = dati.columns[idx]  # Nome originale con maiuscole/minuscole
            medie_riferimento[gruppo] = dati[col_name].mean()
    
    # Identifico il gruppo di base
    gruppo_base = list(medie_riferimento.keys())[0]
    fattore_base = medie_riferimento[gruppo_base]
    
    # Allinea tutti i gruppi alla scala del gruppo base
    dati_allineati = {}
    for gruppo, dati in dati_gruppi.items():
        if gruppo in medie_riferimento:
            fattore_scala = fattore_base / medie_riferimento[gruppo]
            dati_allineati[gruppo] = dati.copy()
            
            # Scala tutte le colonne tranne eventuali colonne non numeriche
            for col in dati.columns:
                if pd.api.types.is_numeric_dtype(dati[col]):
                    dati_allineati[gruppo][col] = dati[col] * fattore_scala
    
    return dati_allineati

# Allinea i dati
dati_allineati = allinea_scale(dati_gruppi)


In [ ]:
# 3. Gestione dei periodi temporali diversi
# In caso i file abbiano periodi diversi, dobbiamo trovare il periodo comune

def trova_periodo_comune(dati_gruppi):
    date_inizio = []
    date_fine = []
    
    for gruppo, df in dati_gruppi.items():
        date_inizio.append(df.index.min())
        date_fine.append(df.index.max())
    
    # Il periodo comune va dalla data di inizio più recente alla data di fine più antica
    inizio_comune = max(date_inizio)
    fine_comune = min(date_fine)
    
    print(f"Periodo comune identificato: {inizio_comune} a {fine_comune}")
    
    # Verifica che il periodo comune sia valido
    if inizio_comune >= fine_comune:
        print("⚠️ Attenzione: Non c'è un periodo comune tra tutti i gruppi!")
        return None, None
    
    return inizio_comune, fine_comune

inizio_comune, fine_comune = trova_periodo_comune(dati_allineati)


In [ ]:
# 4. Filtra i dati per il periodo comune e combina in un unico dataset
def combina_dati(dati_allineati, inizio, fine, query_riferimento="inflazione"):
    if inizio is None or fine is None:
        print("Impossibile combinare i dati: periodo comune non valido")
        return None
    
    # Inizializza il dataframe combinato
    df_combinato = pd.DataFrame(index=pd.date_range(start=inizio, end=fine, freq='MS'))
    
    for gruppo, df in dati_allineati.items():
        # Filtra per il periodo comune
        df_periodo = df[(df.index >= inizio) & (df.index <= fine)]
        
        # Rinomina le colonne per identificare il gruppo
        for col in df_periodo.columns:
            # Evita duplicati della query di riferimento
            if col.lower() != query_riferimento.lower() or gruppo == list(dati_allineati.keys())[0]:
                nome_colonna = f"{gruppo}_{col}"
                # Assicurati che l'indice corrisponda
                if df_periodo.index.equals(df_combinato.index):
                    df_combinato[nome_colonna] = df_periodo[col]
                else:
                    # Ricampiona se necessario (questo dipende dalla frequenza dei tuoi dati)
                    df_temp = df_periodo.reindex(df_combinato.index, method='nearest')
                    df_combinato[nome_colonna] = df_temp[col]
    
    return df_combinato

# Combina i dati
df_gt_combinato = combina_dati(dati_allineati, inizio_comune, fine_comune)



In [ ]:
# 5. Standardizzazione delle serie
def standardizza_serie(df):
    df_std = df.copy()
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df_std[col] = (df[col] - df[col].mean()) / df[col].std()
    
    return df_std

df_gt_std = standardizza_serie(df_gt_combinato)



In [ ]:
# 6. Creazione degli indici PCA per gruppo
# Dobbiamo organizzare le colonne per gruppo (già fatto dal prefisso nel nome colonna)

def crea_indici_pca_per_gruppo(df_std, nomi_gruppi):
    indici_pca = pd.DataFrame(index=df_std.index)
    
    for nome_gruppo in nomi_gruppi.values():
        # Filtra colonne di questo gruppo
        colonne_gruppo = [col for col in df_std.columns if col.startswith(f"{nome_gruppo}_")]
        
        if len(colonne_gruppo) > 1:  # PCA richiede almeno 2 variabili
            # Applica PCA
            pca = PCA(n_components=1)
            componente = pca.fit_transform(df_std[colonne_gruppo])
            
            # Salva il primo componente principale come indice del gruppo
            indici_pca[f'indice_{nome_gruppo}'] = componente.flatten()
            
            # Stampa la varianza spiegata
            print(f"Gruppo {nome_gruppo}: {pca.explained_variance_ratio_[0]:.2%} della varianza spiegata")
            
            # Stampa i loadings (pesi) per interpretazione
            print("Loadings:")
            for i, col in enumerate(colonne_gruppo):
                print(f"  {col}: {pca.components_[0][i]:.4f}")
        
        elif len(colonne_gruppo) == 1:
            # Se c'è una sola colonna, usala direttamente
            indici_pca[f'indice_{nome_gruppo}'] = df_std[colonne_gruppo[0]]
            print(f"Gruppo {nome_gruppo}: una sola colonna disponibile, usata direttamente")
    
    return indici_pca

# Crea indici per ogni gruppo
indici_pca_gruppi = crea_indici_pca_per_gruppo(df_gt_std, nomi_gruppi)



In [ ]:
# 7. Creazione dell'indice globale
pca_globale = PCA(n_components=1)
indice_globale = pca_globale.fit_transform(indici_pca_gruppi)
indici_pca_gruppi['indice_globale'] = indice_globale

# Visualizza gli indici
plt.figure(figsize=(12, 8))
for col in indici_pca_gruppi.columns:
    plt.plot(indici_pca_gruppi.index, indici_pca_gruppi[col], label=col)
plt.legend()
plt.title('Indici Google Trends per Gruppo')
plt.xlabel('Data')
plt.ylabel('Valore Standardizzato')
plt.grid(True)
plt.tight_layout()
plt.show()

# 8. Salva gli indici
indici_pca_gruppi.to_csv('indici_google_trends.csv')
print("Indici salvati con successo in 'indici_google_trends.csv'")
